# Calidad de Datos: Métricas y Dimensiones

Este notebook demuestra cómo medir y calcular métricas de calidad de datos.

**Referencia:** [Calidad de Datos](../calidad/calidad-de-datos.md)

**Objetivos:**
- Calcular dimensiones de calidad
- Visualizar métricas
- Identificar problemas de calidad

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Librerías importadas")

## 2. Crear datos de ejemplo con problemas

In [ ]:
# Datos con problemas de calidad
np.random.seed(42)
n = 1000

df = pd.DataFrame({
    'id': range(1, n + 1),
    'nombre': [f'Usuario {i}' if i % 10 != 0 else None for i in range(1, n + 1)],
    'email': [f'user{i}@email.com' if i % 15 != 0 else None for i in range(1, n + 1)],
    'edad': [np.random.randint(18, 80) if i % 20 != 0 else np.random.randint(150, 200) for i in range(1, n + 1)],
    'salario': [np.random.uniform(30000, 100000) if i % 25 != 0 else -1000 for i in range(1, n + 1)],
    'fecha_registro': pd.date_range('2020-01-01', periods=n, freq='D')
})

# Agregar algunos duplicados
df = pd.concat([df, df.sample(50)], ignore_index=True)

print(f"✅ Datos de ejemplo creados: {len(df)} registros")
print("\nProblemas incluidos:")
print("  - Valores nulos (10% en nombre, 6.7% en email)")
print("  - Valores fuera de rango (edad > 120)")
print("  - Valores negativos (salario)")
print("  - Duplicados")
df.head()

## 3. Calcular Completitud (Completeness)

In [ ]:
def calcular_completitud(df):
    """Calcula completitud por columna."""
    total_filas = len(df)
    completitud = {}
    
    for col in df.columns:
        filas_no_nulas = df[col].notna().sum()
        porcentaje = (filas_no_nulas / total_filas) * 100
        completitud[col] = porcentaje
    
    return pd.Series(completitud)

completitud = calcular_completitud(df)
print("=== COMPLETITUD POR COLUMNA ===")
print(completitud.round(2))

# Visualizar
plt.figure(figsize=(10, 6))
completitud.sort_values(ascending=True).plot(kind='barh', color='steelblue')
plt.axvline(x=95, color='green', linestyle='--', label='Umbral Excelente (95%)')
plt.axvline(x=80, color='orange', linestyle='--', label='Umbral Aceptable (80%)')
plt.xlabel('Completitud (%)')
plt.title('Completitud por Columna')
plt.legend()
plt.tight_layout()
plt.show()

## 4. Calcular Exactitud (Accuracy)

In [ ]:
def calcular_exactitud(df):
    """Calcula exactitud basada en rangos válidos."""
    exactitud = {}
    
    # Edad: debe estar entre 0 y 120
    if 'edad' in df.columns:
        edad_valida = df['edad'].between(0, 120).sum()
        exactitud['edad'] = (edad_valida / len(df)) * 100
    
    # Salario: debe ser positivo
    if 'salario' in df.columns:
        salario_valido = (df['salario'] > 0).sum()
        exactitud['salario'] = (salario_valido / len(df)) * 100
    
    return pd.Series(exactitud)

exactitud = calcular_exactitud(df)
print("=== EXACTITUD (Valores en rango válido) ===")
print(exactitud.round(2))

## 5. Calcular Unicidad (Uniqueness)

In [ ]:
def calcular_unicidad(df, columnas_unicas):
    """Calcula unicidad para columnas que deben ser únicas."""
    unicidad = {}
    
    for col in columnas_unicas:
        if col in df.columns:
            total = len(df)
            unicos = df[col].nunique()
            porcentaje = (unicos / total) * 100
            unicidad[col] = porcentaje
    
    return pd.Series(unicidad)

unicidad = calcular_unicidad(df, ['id', 'email'])
print("=== UNICIDAD ===")
print(unicidad.round(2))

# Detectar duplicados
duplicados = df.duplicated().sum()
print(f"\nDuplicados completos: {duplicados}")
duplicados_email = df.duplicated(subset=['email']).sum()
print(f"Duplicados en email: {duplicados_email}")

## 6. Dashboard de Calidad

In [ ]:
# Crear dashboard de calidad
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Completitud
completitud.sort_values(ascending=True).plot(kind='barh', ax=axes[0, 0], color='steelblue')
axes[0, 0].axvline(x=95, color='green', linestyle='--', alpha=0.5)
axes[0, 0].set_title('Completitud (%)')
axes[0, 0].set_xlabel('%')

# Exactitud
exactitud.plot(kind='bar', ax=axes[0, 1], color='darkgreen')
axes[0, 1].set_title('Exactitud (%)')
axes[0, 1].set_ylabel('%')
axes[0, 1].tick_params(axis='x', rotation=45)

# Unicidad
unicidad.plot(kind='bar', ax=axes[1, 0], color='orange')
axes[1, 0].set_title('Unicidad (%)')
axes[1, 0].set_ylabel('%')
axes[1, 0].tick_params(axis='x', rotation=45)

# Resumen
resumen = pd.DataFrame({
    'Métrica': ['Completitud Promedio', 'Exactitud Promedio', 'Unicidad Promedio'],
    'Valor': [completitud.mean(), exactitud.mean(), unicidad.mean()]
})
resumen.plot(x='Métrica', y='Valor', kind='bar', ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Resumen de Calidad')
axes[1, 1].set_ylabel('%')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 7. Reporte de Calidad

In [ ]:
print("=" * 60)
print("REPORTE DE CALIDAD DE DATOS")
print("=" * 60)
print(f"\nTotal de registros: {len(df)}")
print(f"\nCompletitud promedio: {completitud.mean():.2f}%")
print(f"Exactitud promedio: {exactitud.mean():.2f}%")
print(f"Unicidad promedio: {unicidad.mean():.2f}%")
print(f"\nDuplicados: {duplicados}")
print(f"Valores nulos totales: {df.isnull().sum().sum()}")
print("\n" + "=" * 60)
print("\n✅ Calidad Excelente: > 95%")
print("⚠️  Calidad Aceptable: 80-95%")
print("❌ Calidad Crítica: < 80%")
print("=" * 60)